### Background

My husband Stuart and I worked together to create this first draft interactive map of American Community Survey Census Tracts (2018 data courtesy of Colorado Information Marketplace) that looks at the index of Black / African American (hereafter "B/AA") Household frequency among census blocks. If you're unfamiliar with indexing, the goal is to identify higher concentrations of some focus within the data ignoring the scale at which you are looking. 

*For example: 
Census block A has 500 total inhabitants and 50 are B/AA
Census block B has 100 total inhabitants and 40 are B/AA.*


*Among these two blocks (600 total inhabitants, 90 are B/AA). The index for block A will be 50/500 (0.10) over (90/600), equaling 0.66. The index for block B is (40/100) / (90/600) = 2.66. Even though block B is smaller than A, there is a higher concentration B/AA inhabitants in block b.*





In the map below, different layers exist for ranges of indices from 1.01X up to the max around ~15.9x the average concentration of B/AA inhabitants among the whole state. Each layer can be toggled using the small layer pane in the upper right corner. 

![Layer Toggles](../../references/map_layer_toggle.JPG)

The individual markers throughout the map are healthcare locations likely to be providing care for COVID-19 cases (limited to those inside 1x or more B/AA blocks). Click on a marker to see the name, phone, and if they accept Medicaid (y/n).

![Tooltips](../../references/tooltip.JPG)

-------

In [7]:
import geopandas as gpd
import descartes
import folium
from folium.plugins import MarkerCluster
import branca.colormap as cm
import pandas as pd

In [2]:
#read in geoJSON data for the ACS blocks and healthcare locations
hc_df = gpd.read_file('../../data/output/healthcare.geojson')
ACS_df = gpd.read_file('../../data/output/ACS.geojson')

# high_indexing = gpd.read_file('../../data/output/black_aa_indexed.geojson')
hc_df_clipped = gpd.read_file('../../data/output/clipped_healthcare.geojson')

#read in polygons of black / aa households indexed in colorado
index_101_500 = gpd.read_file('../../data/output/index_101_500.geojson')
index_500_1000 = gpd.read_file('../../data/output/index_500_1000.geojson')
index_1000_1300 = gpd.read_file('../../data/output/index_1000_1300.geojson')
index_1301_plus = gpd.read_file('../../data/output/index_1301_plus.geojson')

In [3]:
#launch OSM basemap focused on colorado at state level
map_osm = folium.Map(location=[39.3324, -105.1420], zoom_start=7)

#add highest indexing polygons
folium.GeoJson(index_1301_plus.to_json(),
               name='13x (or more) concentration of Black/AA households relative to Overall Colorado',
               tooltip=folium.features.GeoJsonTooltip(
                   fields=['pop', 'black_indexed',
                           'pct_black'], localize=True)).add_to(map_osm)
#add 2nd highest indexing polygons

folium.GeoJson(index_1000_1300.to_json(),
               name='10x to 13x concentration of Black/AA households',
               tooltip=folium.features.GeoJsonTooltip(
                   fields=['pop', 'black_indexed',
                           'pct_black'], localize=True)).add_to(map_osm)

#add 3rd highest indexing polygons
folium.GeoJson(index_500_1000.to_json(),
               name='5x to 10x concentration of Black/AA households',
               tooltip=folium.features.GeoJsonTooltip(
                   fields=['pop', 'black_indexed',
                           'pct_black'], localize=True)).add_to(map_osm)

#add 4th highest indexing polygons
folium.GeoJson(index_101_500.to_json(),
               name='1.01x to 5x concentration of Black/AA households',
               tooltip=folium.features.GeoJsonTooltip(
                   fields=['pop', 'black_indexed',
                           'pct_black'], localize=True)).add_to(map_osm)

# use a clustered view of markers
marker_cluster = MarkerCluster().add_to(map_osm)

# add a marker for every record in the filtered data
for each in hc_df_clipped.iterrows():
    folium.Marker(
        [each[1]['Latitude'], each[1]['Longitude']],
        popup=str(each[1]['FAC_NAME'] + '\nPhone: ' + each[1]['PHONE'] +
                  '\nMedicaidAccept: ' + each[1]['MEDICAID']),
        clustered_marker=True).add_to(marker_cluster)

folium.LayerControl().add_to(map_osm)


map_osm

In [4]:
html_string = map_osm.get_root().render()
with open("../../data/output/map_html.html", "w") as text_file:
    print(html_string, file=text_file)

### Choropleth Mapping


In [12]:
#define linear color map using folium
linear = cm.LinearColormap(['white', 'blue'], vmin=0, vmax=15)

linear



In [ ]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': step(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)